In [1]:
import pandas as pd
import pandasql as ps
import time

In [2]:
user_usage = pd.read_csv("./data/user_usage.csv")
user_device = pd.read_csv("./data/user_device.csv")

In [3]:
user_usage.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


In [4]:
user_device.head()

,use_id,user_id,platform,platform_version,device,use_type_id
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3
2,22784,28473,android,5.1,SM-G903F,1
3,22785,15200,ios,10.2,"iPhone7,2",3
4,22786,28239,android,6.0,ONE E1003,1


Cоеденим таблицы user_usage c user_device по uder_id 

In [5]:
start_time = time.time()
result = pd.merge(user_usage, 
                  user_device[['use_id', 'platform', 'device']], on='use_id')
print("{} second".format(round((time.time()-start_time), 4)))
result.head()

0.004 second


,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


In [6]:
start_time = time.time()
result_sql = ps.sqldf("SELECT u.*, d.platform, d.device FROM user_usage u INNER JOIN user_device d ON (u.use_id=d.use_id);")
print("{} second".format(round((time.time()-start_time), 4)))
result.head()

0.357 second


,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


Найдем средние значения параметров использования телефонов 

In [9]:
start_time = time.time()
aggr_func_result = result.groupby('device').agg({"outgoing_mins_per_month": "mean",
        "outgoing_sms_per_month": "mean",
        "monthly_mb": "mean",
        "use_id": "count"})
print("{} second".format(round((time.time()-start_time), 4)))
print(aggr_func_result)

0.004 second
                        outgoing_mins_per_month  outgoing_sms_per_month  \
device                                                                    
A0001                                170.395000               62.100000   
C6603                                 92.520000              162.390000   
D2303                                 96.845000               35.375000   
D5503                                146.450000               48.670000   
D5803                                244.880000              105.950000   
D6603                                362.010000               14.190000   
E6653                                135.090000               42.020000   
EVA-L09                              115.260000                0.920000   
F3111                                 46.262500                0.415000   
GT-I8190N                             85.970000               26.940000   
GT-I9195                             199.430000               44.126667   
GT-I9300    

In [8]:
start_time = time.time()
aggr_func_result_sql = ps.sqldf("SELECT avg(outgoing_sms_per_month), avg(monthly_mb), COUNT(use_id) FROM result_sql;")
print("{} second".format(round((time.time()-start_time), 4)))
print(aggr_func_result)

0.0325 second
                        outgoing_mins_per_month  outgoing_sms_per_month  \
device                                                                    
A0001                                170.395000               62.100000   
C6603                                 92.520000              162.390000   
D2303                                 96.845000               35.375000   
D5503                                146.450000               48.670000   
D5803                                244.880000              105.950000   
D6603                                362.010000               14.190000   
E6653                                135.090000               42.020000   
EVA-L09                              115.260000                0.920000   
F3111                                 46.262500                0.415000   
GT-I8190N                             85.970000               26.940000   
GT-I9195                             199.430000               44.126667   
GT-I9300   

Вывод: время обработки запроса pandasql в разы больше pandas